In [23]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)
sys.path

['c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\notebooks',
 'C:\\ProgramData\\Anaconda3\\python38.zip',
 'C:\\ProgramData\\Anaconda3\\DLLs',
 'C:\\ProgramData\\Anaconda3\\lib',
 'C:\\ProgramData\\Anaconda3',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env',
 '',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env\\lib\\site-packages',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env\\lib\\site-packages\\win32',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\src\\utils',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\src\

In [24]:
sys.path

['c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\notebooks',
 'C:\\ProgramData\\Anaconda3\\python38.zip',
 'C:\\ProgramData\\Anaconda3\\DLLs',
 'C:\\ProgramData\\Anaconda3\\lib',
 'C:\\ProgramData\\Anaconda3',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env',
 '',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env\\lib\\site-packages',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env\\lib\\site-packages\\win32',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\elmy_env\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\src\\utils',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy',
 'c:\\Users\\emac\\Documents\\MachineLearningAvance\\elmy\\elmy\\src\

In [25]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from process_data import process_data
from weighted_accuracy import weighted_accuracy_scorer
from plot_learning_curves import plot_learning_curves


In [26]:
X = os.path.join("..", "data","original_input", "X_train_Wwou3IE.csv")
Y = os.path.join("..", "data","original_input", "X_test_GgyECq8.csv")
W = os.path.join("..", "data","original_input", "y_train_jJtXgMX.csv")

X_train = pd.read_csv(X, delimiter=',')
X_test = pd.read_csv(Y,delimiter=',')
y_train = pd.read_csv(W, delimiter=',')

In [27]:
X_train = process_data(X_train.copy(deep=True), "predicted_spot_price", None, "standard")
X_test = process_data(X_train.copy(deep=True), "predicted_spot_price", None, "standard")
y_train = process_data(X_train.copy(deep=True), None, None, None)

In [28]:
X_train

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std
DELIVERY_START,,,,,,,,
2022-01-01 01:00:00+00:00,-0.405599,1.143744,0.049156,1.145111,-0.441788,-0.709854,-0.349637,-0.580887
2022-01-01 02:00:00+00:00,-0.682678,1.143744,0.049156,1.145111,-0.409506,-0.709854,-0.482002,-0.580887
2022-01-01 03:00:00+00:00,-0.810713,1.143744,0.049156,1.145111,-0.366164,-0.709854,-0.614471,-0.580887
2022-01-01 04:00:00+00:00,-0.846578,1.143744,0.049156,1.145111,-0.318637,-0.709854,-0.676317,-0.580887
2022-01-01 05:00:00+00:00,-0.798790,1.143744,0.049156,1.145111,-0.249289,-0.709854,-0.715292,-0.580887
...,...,...,...,...,...,...,...,...
2023-03-29 17:00:00+00:00,-0.275482,1.143744,1.010389,0.384129,0.908396,-0.469079,0.924332,-0.399128
2023-03-29 18:00:00+00:00,-0.293084,1.143744,1.010389,0.384129,1.143341,-0.669539,0.230275,-0.522000
2023-03-29 19:00:00+00:00,-0.522752,1.143744,1.010389,0.384129,1.375595,-0.709854,0.011252,-0.580887


In [29]:
month_to_choose = [4,5,6,7,8,9,10]

print("month_to_choose:", month_to_choose)
X_train = X_train.loc[X_train.index.month.isin(month_to_choose)]
y_train = y_train.loc[y_train.index.month.isin(month_to_choose)]
print("Filtered X_train:", X_train)

month_to_choose: [4, 5, 6, 7, 8, 9, 10]
Filtered X_train:                            load_forecast  coal_power_available  \
DELIVERY_START                                                   
2022-04-01 07:00:00+00:00       1.185520             -0.027838   
2022-04-01 08:00:00+00:00       1.220723             -0.027838   
2022-04-01 09:00:00+00:00       1.221385             -0.027838   
2022-04-01 10:00:00+00:00       1.284788             -0.027838   
2022-04-01 11:00:00+00:00       1.143788             -0.027838   
...                                  ...                   ...   
2022-10-31 19:00:00+00:00      -0.977736             -0.027838   
2022-10-31 20:00:00+00:00      -1.215922             -0.027838   
2022-10-31 21:00:00+00:00      -1.160279             -0.027838   
2022-10-31 22:00:00+00:00      -1.130659             -0.027838   
2022-10-31 23:00:00+00:00      -1.463475             -0.027838   

                           gas_power_available  nucelear_power_available  \
DELIVER

In [30]:
month_to_choose = [4,5,6,7,8,9,10]
month_to_choose = [pd.Timestamp(year=2022, month=month, day=1) for month in month_to_choose]

print("month_to_choose:", month_to_choose)
X_train = X_train.loc[X_train.index.isin(month_to_choose)]
y_train = y_train.loc[y_train.index.isin(month_to_choose)]
print("Filtered X_train:", X_train)

month_to_choose: [Timestamp('2022-04-01 00:00:00'), Timestamp('2022-05-01 00:00:00'), Timestamp('2022-06-01 00:00:00'), Timestamp('2022-07-01 00:00:00'), Timestamp('2022-08-01 00:00:00'), Timestamp('2022-09-01 00:00:00'), Timestamp('2022-10-01 00:00:00')]
Filtered X_train: Empty DataFrame
Columns: [load_forecast, coal_power_available, gas_power_available, nucelear_power_available, wind_power_forecasts_average, solar_power_forecasts_average, wind_power_forecasts_std, solar_power_forecasts_std]
Index: []


In [31]:
print(X_train.index.unique())

month_to_choose = [4,5,6,7,8,9,10]
# Convert month_to_choose to a set for easy comparison
month_to_choose_set = set(month_to_choose)

# Check if any timestamps in month_to_choose are present in X_train index
matching_timestamps = [timestamp for timestamp in month_to_choose_set if timestamp in X_train.index]

print("Matching timestamps between month_to_choose and X_train index:")
print(matching_timestamps)

DatetimeIndex([], dtype='datetime64[ns, UTC]', name='DELIVERY_START', freq=None)
Matching timestamps between month_to_choose and X_train index:
[]


In [32]:
X_train = X_train.loc[X_train.index.isin(matching_timestamps)]
y_train = y_train.loc[y_train.index.isin(matching_timestamps)]

In [33]:
X_train

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std
DELIVERY_START,,,,,,,,
